In [2]:
import os
import re
import yaml
import json
import ast
import praw
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

from textwrap import dedent
from reddit_helper import *    

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

from pydantic import BaseModel

In [3]:
api_file_path = 'api_keys.json'
with open(api_file_path, 'r') as file:
    api_keys = json.load(file)        
openai_gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])
openai_gpt4 = ChatOpenAI(model_name="gpt-4", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])        
llama31_8b = Ollama(model='llama3.1:8b',)
gemma2_9b = Ollama(model='gemma2:9b',)

In [4]:
cfg_file_path = 'casaai_config.yaml'
with open(cfg_file_path, 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)
product_long = cfg.get('product_long_description', '')      
product_short = cfg.get('product_short_description', '')

In [42]:
def re_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'rank':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, rank = matches[0]
    else:
        comment_id = "na"
        rank = 0

    justification = input_string.split("justification")[-1]
    
    return comment_id, rank, justification

In [56]:
backstory = "You are a content analyst with expertise in analyzing web content and \
             extracting relevant information. \
             - You ensures that content is relevant, high-quality, and aligned with the marketing of \
             product. \
             - You evaluate content by analyzing user interactions, such as likes, \
             comments, and views. \
             - Also consider factors such as keyword density, context accuracy, and user intent. \
             - You identifies content that effectively meets audience expectations, flags irrelevant material, and \
            provides insights to enhance content targeting."
goal = "Analyze web content and extract relevant information, Accurately assess the impact and effectiveness of \
        content, Ensure that content is highly pertinent and aligned with the marketing of {product_short}."
content_analysis_agent = Agent(
                            role="Content Analyst",
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [57]:
class DecisionOutput(BaseModel):
    comment_id: str
    decision: str
    justification: str

In [58]:
tip_text = "If you do your BEST WORK, I'll give you a $10,000 commission!"

In [59]:
descr = "{comment_id} is the id of the comment that you need to provide decision and justification. You will also be \
provided data belonging to this comment and also its parent comments. You will get these data in {input_data}. You should: \
- Analyze this and determine relevance of {comment_id} based on identified keywords and phrases w.r.to the marketing of {product_long}. \
- Evaluate the level of user interaction in {comment_id} by analyzing the provided content. This includes analyzing metrics such as \
likes, and views.\
- Assess how well the {comment_id} aligns with product by analyzing the provided content.\
- Based on these, make a decision whether comment is relevant for markeing of product or not.\
- If relevant populate 'relevant' against 'decision' key of json output, else pass 'discard'. \
- Comments that only acknowledges or expresses gratitude should be marked irrelvant as they dont have any content.{tip_text}"

expected_out = "comment id, decision and a brief justification (less than 15 words) \
                explaining the rationale behind the decision" 
        
content_analysis_task = Task(
                            description=descr,
                            expected_output=expected_out,
                            output_json=DecisionOutput,
                            agent=content_analysis_agent,
                            )

In [60]:
response_creation_crew = Crew(
    agents=[content_analysis_agent,],
    tasks=[content_analysis_task,],
    verbose=False,
)

2024-08-20 10:59:58,919 - 139704650090304 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [14]:
reddit_posts, reddit_post_ids = fetch_reddit_test()
condensed_reddit_data, unique_post_ids, unique_comment_ids = condense_data(reddit_posts, reddit_post_ids)

post_cnt: 0 + 10 = 10
comm_cnt: 0 + 560 = 560
cond_cnt: 570 = 10 + 560
Cross_ck: 10 = 10


In [61]:
comment_dict = {}
tot_data = 0
for idx1, item in enumerate(condensed_reddit_data):
    for idx2, data in enumerate(item):
        tot_data += 1
        c_id = data['comment_id']
        p_id = data['parent_id']
        text = data['text']
        comment_dict[c_id] = {'comment_id': c_id, 'parent_id': p_id, 'text': text}
    print(idx1, ':', idx2)
print(len(comment_dict), '=', tot_data)

0 : 169
1 : 108
2 : 31
3 : 57
4 : 0
5 : 33
6 : 67
7 : 60
8 : 1
9 : 34
570 = 570


In [62]:
def get_data_details(comm_id):
    done = 0
    data_details = []
    while done==0:
        if comm_id in comment_dict:
            details = comment_dict[comm_id]
            data_details.append(details)
            comm_id = details['parent_id']
        else:
            done = 1
    return data_details

In [64]:
decision_lst = []
for idx, reddit_data_item in enumerate(condensed_reddit_data):
    comment_lst = []
    for idx2, comment_data in enumerate(reddit_data_item):        
        comm_id = comment_data['comment_id']        
        print(f'STARTING {idx} - {idx2} - {comm_id}')
        data_details = get_data_details(comm_id)      
        input_dict = {"comment_id": comm_id,
                      "input_data": data_details,
                      "product_long": product_long,
                      "product_short": product_short,
                      "tip_text":tip_text}
        scoring_result = response_creation_crew.kickoff(inputs=input_dict)
        try:
            json_out = ast.literal_eval(scoring_result.json)
        except:
            print('Exception in json - trying re')
            _, d, j = re_string(scoring_result.raw)            
            json_out = {'comm_id': comm_id, 'decision': str(d), 'justification': str(j)}
        comment_lst.append(json_out)
    decision_lst.append(comment_lst)

STARTING 0 - 0 - kgesxn
 Error parsing JSON: Expecting property name enclosed in double quotes: line 2 column 3 (char 5). Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for DecisionOutput
  Invalid JSON: key must be a string at line 2 column 3 [type=json_invalid, input_value="{\n  'comment_id': 'kges...h CasaAI's purpose.'\n}", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid. The JSON structure doesn't match the expected model. Attempting alternative conversion method.
STARTING 0 - 1 - ggfdsvg
 Error parsing JSON: Expecting property name enclosed in double quotes: line 2 column 2 (char 3). Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for DecisionOutput
  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value="{'comment_id': 'ggfdsvg'...t relation to CasaAI.'}", input_type=str]
    For further information visit https://errors.pydantic.de

In [68]:
len(decision_lst), len(decision_lst[0]), decision_lst[0][0]

(10,
 170,
 {'comment_id': 'kgesxn',
  'decision': 'relevant',
  'justification': "Showcases home transformation, aligns with CasaAI's purpose."})

In [96]:
analysis_lst = []
bad_list = ['user123','str']
for item in decision_lst:
    tot = 0
    rel = 0
    disc = 0   
    bad_cnt = 0
    sep = "^"
    for comment in item:
        tot += 1
        if comment['decision'] == 'relevant':
            rel += 1
        else:
            disc += 1
        comm_key = list(comment.keys())[0]
        comm_id  = comment[comm_key]
        if comm_id in bad_list:
            bad_cnt += 1
        else:
            parent_id = comment_dict[comm_id]["parent_id"]
            if parent_id is None:
                parent_id = 'NA'
            decision = comment['decision']
            justify  = comment["justification"]
            comment_text = comment_dict[comm_id]["text"]
            text_str = sep + comm_id + sep + parent_id + sep + decision + sep + justify + sep + comment_text + sep
            analysis_lst.append(text_str)
    print(f'tot : {tot}, rel : {rel}, discarded : {disc}, bad_cnt : {bad_cnt} analysis : {len(analysis_lst)}')

tot : 170, rel : 113, discarded : 57, bad_cnt : 0 analysis : 170
tot : 109, rel : 97, discarded : 12, bad_cnt : 0 analysis : 279
tot : 32, rel : 25, discarded : 7, bad_cnt : 0 analysis : 311
tot : 58, rel : 44, discarded : 14, bad_cnt : 0 analysis : 369
tot : 1, rel : 1, discarded : 0, bad_cnt : 0 analysis : 370
tot : 34, rel : 23, discarded : 11, bad_cnt : 0 analysis : 404
tot : 68, rel : 44, discarded : 24, bad_cnt : 1 analysis : 471
tot : 61, rel : 52, discarded : 9, bad_cnt : 0 analysis : 532
tot : 2, rel : 2, discarded : 0, bad_cnt : 0 analysis : 534
tot : 35, rel : 26, discarded : 9, bad_cnt : 1 analysis : 568


In [74]:
comment_dict['kgesxn']

{'comment_id': 'kgesxn',
 'parent_id': None,
 'text': 'Our newly renovated living room! This room has seen a huge transformation! If you’d like to see more then follow our Instagram @houseroundthebend'}

In [95]:
analysis_lst[100]

"^^ggg12zp^^ggg0jqv^^relevant^^Provides room dimensions and future plans aligning with CasaAI's use.Thank you! That rug is actually 3 metres long and about 2 metres wide! I just didn’t get it in the picture! Art to follow of course, the walls are looking a little sparse!^^"

In [97]:
with open('decision_result_v4_0820.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in analysis_lst:
        file.write(item + "\n")  # Add a newline character after each string

print(f"List has been written")

List has been written


In [98]:
with open('decision_result_v4_0820.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

739


In [34]:
# For each post-comment combo select the post-comments whose score >= 7

In [ ]:
# Then select a threshold like 20% or top 3 whichever is bigger of comments to craft response

In [ ]:
# Build response for these comments With Manager LLM + only 1 response writer + meta reviewer